# Validation of Hamiltonian generation

$\newcommand{\ket}[1]{|#1\rangle}$
$\newcommand{\bra}[1]{\langle#1|}$
$\newcommand{\ketbra}[2]{| #1 \rangle\langle #2 |}$

In [1]:
import numpy as np
import qudit_sim
import qudit_sim.visualization

## Single-qutrit Hamiltonian
### No drive, qudit frame

In [2]:
hgen_nodrive = qudit_sim.HamiltonianBuilder(3)
hgen_nodrive.add_qudit(qubit_frequency=100., anharmonicity=-8., drive_amplitude=6., qudit_id='q0')
hgen_nodrive.build()

[]

By default, frame frequencies are set to free-qudit level gaps (qudit frame). Single qudit with no drive will therefore evolve with a null Hamiltonian.

### No drive, lab frame
If we move to the "lab frame":

In [3]:
hgen_nodrive.set_global_frame('lab')
hamiltonian = hgen_nodrive.build()
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

The Hamiltonian has a single term (free Hamiltonian).

### Constant drive, lab frame

Adding a constant drive

In [4]:
hgen = qudit_sim.HamiltonianBuilder(3)
hgen.add_qudit(qubit_frequency=100., anharmonicity=-8., drive_amplitude=6., qudit_id='q0')
hgen.add_drive('q0', frequency=99., amplitude=0.5+0.5j)

In [5]:
hgen.set_global_frame('lab')
hamiltonian = hgen.build(rwa=False)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

This is an off-resonant drive for both 0-1 and 1-2 transitions. Since we are currently in the lab frame, we expect
- No difference in the static Hamiltonian (no static contribution from drive terms)
- Creation / annihilation operators multiplied by the drive amplitude (6) times amplitude (0.5)

The second set of terms arise from

$$
H_{d} = \Omega_j \left( p_j(t) \cos (\nu_j t) + q_j(t) \sin (\nu_j t) \right) \left( b_j^{\dagger} + b_j \right),
$$

where $b_j^{\dagger} + b_j = \ketbra{0}{1} + \ketbra{1}{0} + \sqrt{2} \ketbra{1}{2} + \sqrt{2} \ketbra{2}{1}.$

### Constant drive, qudit frame

If we move back to the qudit frame,

$$
\begin{gather}
H_{\mathrm{static}} = 0 \\
\tilde{H}_{d} = \Omega_j \left( p_j(t) \cos (\nu_j t) + q_j(t) \sin (\nu_j t) \right) \left( \tilde{b}_j^{\dagger} + \tilde{b}_j \right),
\end{gather}
$$

where

$$
\tilde{b}_j = e^{-i\omega_j t} \ketbra{0}{1} + \sqrt{2} e^{-i(\omega_j + \Delta_j)t} \ketbra{1}{2},
$$

and therefore

$$
\tilde{b}_j^{\dagger} + \tilde{b}_j = \cos (\omega_j t) (\ketbra{0}{1} + \ketbra{1}{0}) + i\sin (\omega_j t) (-\ketbra{0}{1} + \ketbra{1}{0})+ \sqrt{2} \cos ((\omega_j + \Delta_j) t) (\ketbra{1}{2} + \ketbra{2}{1}) + i\sqrt{2} \sin ((\omega_j + \Delta_j) t) (-\ketbra{1}{2} + \ketbra{2}{1}).
$$

In [6]:
hgen.set_global_frame('qudit')
hamiltonian = hgen.build(rwa=False)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, drive frame

In the drive frame (drive frequency $\nu$),

$$
H_{\mathrm{diag}} = (\omega_j - \nu) \ketbra{1}{1} + (2\omega_j + \Delta_j - 2\nu) \ketbra{2}{2}
$$

and

$$
\tilde{b}_j = e^{-i \nu t} (\ketbra{0}{1} + \sqrt{2}\ketbra{1}{2})
$$

In [7]:
hgen.set_global_frame('drive')
hamiltonian = hgen.build(rwa=False)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, qudit frame, RWA

In the qudit frame with the rotating-wave approximation,

$$
\begin{split}
\bar{H}_{d} = \frac{\Omega_j}{2} & \left[ r^*_j(t) (e^{i(\nu - \omega_j)t} \ketbra{0}{1} + e^{i(\nu - \omega_j - \Delta_j)t} \ketbra{1}{2}) + \mathrm{h.c.} \right] \\
= \frac{\Omega_j}{2} & [(p(t)\cos(\epsilon_0 t) + q(t)\sin(\epsilon_0 t)) (\ketbra{0}{1} + \ketbra{1}{0}) + i (-p(t)\sin(\epsilon_0 t) + q(t)\cos(\epsilon_0 t)) (-\ketbra{0}{1} + \ketbra{1}{0}) \\
& + (p(t)\cos(\epsilon_1 t) + q(t)\sin(\epsilon_1 t)) (\ketbra{1}{2} + \ketbra{2}{1}) + i (-p(t)\sin(\epsilon_1 t) + q(t)\cos(\epsilon_1 t)) (-\ketbra{1}{2} + \ketbra{2}{1})].
\end{split}
$$

In [8]:
hgen.set_global_frame('qudit')
hamiltonian = hgen.build(rwa=True)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, drive frame, RWA

And in the drive frame with RWA,

$$
\begin{align}
H_{\mathrm{diag}} = & (\omega_j - \nu) \ketbra{1}{1} + (2\omega_j + \Delta_j - 2\nu) \ketbra{2}{2} \\
\bar{H}_{d} = & \frac{\Omega_j}{2} [(p(t)-iq(t)) (\ketbra{0}{1} + \sqrt{2}\ketbra{1}{2}) + (p(t) + iq(t)) (\ketbra{1}{0} + \sqrt{2}\ketbra{2}{1})].
\end{align}
$$

But since $p(t) = q(t) = 0.5$ (constant), we get only a single $H_{\mathrm{static}}$.

In [9]:
hgen.set_global_frame('drive')
hamiltonian = hgen.build(rwa=True)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, lab frame with phase offset

Nonzero frame phase offsets in the lab frame

In [10]:
hgen.set_frame('q0', frequency=np.zeros((2,)), phase=np.array([0.1 * np.pi, 0.2 * np.pi]))
hamiltonian = hgen.build(rwa=False)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, qudit frame with phase offset, RWA

In [11]:
hgen.set_frame('q0', frequency=None, phase=np.array([0.1 * np.pi, 0.2 * np.pi]))
hamiltonian = hgen.build(rwa=True)
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

## Two-qutrit Hamiltonian

Using an unrealistically large coupling to make the effects of Hint pronounced

### No drive, qudit frame

In [12]:
hgen_nodrive = qudit_sim.HamiltonianBuilder(3, default_frame='qudit')
hgen_nodrive.add_qudit(qubit_frequency=100., anharmonicity=-8., drive_amplitude=6., qudit_id='q0')
hgen_nodrive.add_qudit(qubit_frequency=110., anharmonicity=-7., drive_amplitude=7., qudit_id='q1')
hgen_nodrive.add_coupling('q0', 'q1', 3.) # 0.03 is a more realistic number

In the qudit frame we have

$$
\begin{split}
\tilde{H}_{\mathrm{int}} & = J_{jk} \left[(e^{i\omega_0 t}\ketbra{1}{0}_0 + \sqrt{2} e^{i(\omega_0 + \Delta_0) t} \ketbra{2}{1}_0) (e^{-i\omega_1 t}\ketbra{0}{1}_1 + \sqrt{2} e^{-i(\omega_1 + \Delta_1) t} \ketbra{1}{2}_1) + \mathrm{h.c.}\right] \\
& = J_{jk} (e^{i(\omega_0 - \omega_1) t}\ketbra{1, 0}{0, 1} + \sqrt{2} e^{i(\omega_0 - \omega_1 - \Delta_1) t} \ketbra{1, 1}{0, 2} + \sqrt{2} e^{i(\omega_0 + \Delta_0 - \omega_1) t} \ketbra{2, 0}{1, 1} + 2 e^{i(\omega_0 + \Delta_0 - \omega_1 - \Delta_1) t} \ketbra{2, 1}{1, 2} + \mathrm{h.c.})
\end{split}
$$

In [13]:
hamiltonian = hgen_nodrive.build()
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### No drive, lab frame

If we move to the lab frame, we will find a fullly static Hamiltonian

In [14]:
hgen_nodrive.set_global_frame('lab')
hamiltonian = hgen_nodrive.build()
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### No drive, dressed frame

In [15]:
hgen_nodrive.set_global_frame('dressed')
hamiltonian = hgen_nodrive.build()
qudit_sim.visualization.print_hamiltonian(hamiltonian)

<IPython.core.display.Latex object>

### Constant drive, qudit frame with phase offset, RWA

In [16]:
hgen = qudit_sim.HamiltonianBuilder(3, default_frame='qudit')
hgen.add_qudit(qubit_frequency=100., anharmonicity=-8., drive_amplitude=6., qudit_id='q0')
hgen.add_qudit(qubit_frequency=110., anharmonicity=-7., drive_amplitude=7., qudit_id='q1')
hgen.add_coupling('q0', 'q1', 0.03)
hgen.add_drive('q0', frequency=110., amplitude=0.5+0.5j)

In [17]:
hgen.set_frame('q0', phase=np.array([0.1, 0.2]))
hgen.set_frame('q1', phase=np.array([0.05, 0.15]))
hamiltonian = hgen.build(rwa=True)
qudit_sim.visualization.print_hamiltonian(hamiltonian, phase_norm=None)

<IPython.core.display.Latex object>